In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
customers = pd.read_csv('./dacon_data/open/customers.csv')
locations = pd.read_csv('./dacon_data/open/locations.csv')
orders = pd.read_csv('./dacon_data/open/orders.csv')
order_items = pd.read_csv('./dacon_data/open/order_items.csv')
payments = pd.read_csv('./dacon_data/open/payments.csv')
products = pd.read_csv('./dacon_data/open/products.csv')
reviews = pd.read_csv('./dacon_data/open/reviews.csv')
sellers = pd.read_csv('./dacon_data/open/sellers.csv')
# temp = pd.merge(reviews,order_items, how='inner',on='Order_id')
# temp = pd.merge(temp,orders, how='inner',on='Order_id')
# temp = pd.merge(temp,payments, how='inner',on='Order_id')
# temp = pd.merge(temp,products, how='inner',on='Product_id')
# temp = pd.merge(temp,customers, how='inner',on='Customer_id')
# temp = pd.merge(temp,sellers, how='inner',on='Seller_id')
# temp = temp.drop(["Review_creation_date","Review_answer_timestamp","Order_status","Payment_sequential","Payment_type","Payment_installments"],axis=1)
# temp = temp.drop(["Product_weight_g","Product_length_cm","Product_height_cm","Customer_zipcode_prefix","Seller_zipcode_prefix"],axis=1)
# temp = temp.drop(["Product_width_cm"],axis=1)

In [3]:
reviews.head(1)

,Review_id,Order_id,Review_score,Review_creation_date,Review_answer_timestamp
0,REVIEW_00000,ORDER_01674,4,2019-01-18 00:00:00,2019-01-18 21:46:59


In [4]:
reviews_groupby = reviews.groupby(["Order_id"])["Review_score"].mean().reset_index()

In [5]:
temp = pd.merge(reviews_groupby,order_items, how='inner',on='Order_id')

In [6]:
temp

,Order_id,Review_score,Order_item_id,Product_id,Seller_id,Price,Freight_value
0,ORDER_00000,4.0,1,PRODUCT_02093,SELLER_0491,29.99,8.72
1,ORDER_00001,5.0,1,PRODUCT_26747,SELLER_2662,45.00,27.20
2,ORDER_00002,5.0,1,PRODUCT_10598,SELLER_1354,19.90,8.72
3,ORDER_00003,4.0,1,PRODUCT_16703,SELLER_1518,147.90,27.36
4,ORDER_00004,5.0,1,PRODUCT_08274,SELLER_1599,59.99,15.17
...,...,...,...,...,...,...,...
99772,ORDER_88084,4.0,1,PRODUCT_01588,SELLER_0031,174.90,20.10
99773,ORDER_88085,5.0,1,PRODUCT_13335,SELLER_1892,205.99,65.02
99774,ORDER_88086,2.0,1,PRODUCT_07372,SELLER_2445,179.99,40.59
99775,ORDER_88086,2.0,2,PRODUCT_07372,SELLER_2445,179.99,40.59


In [7]:
# temp.loc[temp['Order_id']=='ORDER_14054']

In [8]:
temp = pd.merge(temp,orders, how='inner',on='Order_id')
temp = pd.merge(temp,payments, how='inner',on='Order_id')
temp = pd.merge(temp,products, how='inner',on='Product_id')
temp = pd.merge(temp,customers, how='inner',on='Customer_id')
temp = pd.merge(temp,sellers, how='inner',on='Seller_id')

In [9]:
locations_group_lat=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lat"].mean().reset_index()
locations_group_lng=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lng"].mean().reset_index()
locations_group =  pd.merge(locations_group_lat, locations_group_lng, how = 'inner', on="Geolocation_zipcode_prefix")

In [10]:
temp = pd.merge(temp, locations_group, left_on = 'Customer_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
temp = temp.rename(columns={'Geolocation_lat':'Customer_lat',"Geolocation_lng" : "Customer_lng" })

In [11]:
temp = pd.merge(temp, locations_group, left_on = 'Seller_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
temp = temp.rename(columns={'Geolocation_lat':'Seller_lat',"Geolocation_lng" : "Seller_lng" })

In [12]:
temp["Distance"] = ((temp["Customer_lat"] - temp["Seller_lat"])**2 + (temp["Customer_lng"] - temp["Seller_lng"])**2) **(1/2)

In [13]:
temp['Order_purchase_timestamp']=pd.to_datetime(temp['Order_purchase_timestamp'])
temp['Order_delivered_carrier_date']=pd.to_datetime(temp['Order_delivered_carrier_date'])
temp['Order_delivered_customer_date']=pd.to_datetime(temp['Order_delivered_customer_date'])
temp['Order_estimated_delivery_date']=pd.to_datetime(temp['Order_estimated_delivery_date'])

In [14]:
# 구매할때부터 물류처리할때까지 걸리는 시간
temp["time_diff_purchase_carrier"] = temp["Order_delivered_carrier_date"] - temp["Order_purchase_timestamp"]
# 구매할때부터 실제배송할때까지 걸리는 시간
temp["time_diff_purchase_customer"] = temp["Order_delivered_customer_date"] - temp["Order_purchase_timestamp"]
# 물류처리할때부터 실제배송할때까지 걸리는 시간
temp["time_diff_carrier_customer"] = temp["Order_delivered_customer_date"] - temp["Order_delivered_carrier_date"]
# 기대배송날짜와 실제배송날짜의 차이
temp["time_diff_customer_delivery"]= temp['Order_estimated_delivery_date'] - temp["Order_delivered_customer_date"]

In [15]:
temp["time_diff_purchase_carrier_second"] = temp["time_diff_purchase_carrier"].apply(lambda x : x.total_seconds())
temp["time_diff_purchase_customer_second"] = temp["time_diff_purchase_customer"].apply(lambda x : x.total_seconds())
temp["time_diff_carrier_customer_second"] = temp["time_diff_carrier_customer"].apply(lambda x : x.total_seconds())
temp["time_diff_customer_delivery_second"] = temp["time_diff_customer_delivery"].apply(lambda x : x.total_seconds())

In [16]:
temp['Product_weight_g'] = temp['Product_weight_g'].replace('Unknown', np.nan).fillna(0).astype(float)
temp['YearMonth'] = temp['Order_purchase_timestamp'].dt.strftime('%Y%m') #월별로 분류

In [17]:
import pickle
with open("./dacon_total_df.pkl", "wb") as f:
    pickle.dump(temp,f)

In [18]:
# group by 용 Product_category_name                   home_utilities
# 정규화

In [19]:
# temp.columns

In [20]:
# product_id를 가져오자

In [21]:
# 'Payment_value': 1,
#     'Review_score': 0.8,
#     'Price': 0.6,
#     'time_diff_customer_delivery_second': 0.5,
#     'Distance': 0.4,
#     'Freight_value': 0.3,
#     'time_diff_purchase_customer_second': 0.2,

In [22]:
temp.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Review_score,Order_item_id,Price,Freight_value,Payment_sequential,Payment_installments,Payment_value,Product_weight_g,Customer_zipcode_prefix,Seller_zipcode_prefix,Geolocation_zipcode_prefix_x,Customer_lat,Customer_lng,Geolocation_zipcode_prefix_y,Seller_lat,Seller_lng,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second
Review_score,1.000000,-0.136631,0.004038,-0.036344,0.009013,-0.041931,-0.082200,-0.024695,-0.030842,0.030551,-0.030842,-0.037966,-0.042776,0.030551,-0.007809,0.012715,-0.059338,-0.149873,-0.309528,-0.276505,0.243981
Order_item_id,-0.136631,1.000000,-0.060882,-0.026244,-0.000802,0.072410,0.256940,-0.002340,-0.004303,-0.018655,-0.004303,-0.018944,-0.018802,-0.018655,-0.005989,-0.010462,-0.020506,0.048714,-0.016850,-0.038663,0.019393
Price,0.004038,-0.060882,1.000000,0.410490,0.000156,0.279295,0.732875,0.340641,0.041878,0.087132,0.041878,0.055736,0.015998,0.087132,0.024830,0.046537,0.084910,0.063440,0.061040,0.040461,0.001780
Freight_value,-0.036344,-0.026244,0.410490,1.000000,0.009163,0.186932,0.368741,0.615169,0.225188,0.147593,0.225188,0.274120,0.092461,0.147593,0.036002,-0.004605,0.389559,0.093456,0.220277,0.202256,0.037662
Payment_sequential,0.009013,-0.000802,0.000156,0.009163,1.000000,-0.089320,-0.065962,0.028461,-0.031319,0.005551,-0.031319,-0.011271,0.004977,0.005551,0.006480,-0.001422,-0.014118,0.011558,0.000573,-0.004171,-0.000903
Payment_installments,-0.041931,0.072410,0.279295,0.186932,-0.089320,1.000000,0.268191,0.181103,0.056723,0.036800,0.056723,0.059638,0.030472,0.036800,0.009270,-0.010432,0.083342,-0.001386,0.043322,0.047984,0.032023
Payment_value,-0.082200,0.256940,0.732875,0.368741,-0.065962,0.268191,1.000000,0.302633,0.049997,0.067348,0.049997,0.058305,0.015592,0.067348,0.012298,0.025748,0.086942,0.095294,0.059627,0.025690,0.014989
Product_weight_g,-0.024695,-0.002340,0.340641,0.615169,0.028461,0.181103,0.302633,1.000000,0.001849,0.011210,0.001849,0.002381,0.001332,0.011210,-0.022212,0.011481,-0.008067,0.148066,0.081033,0.027208,-0.011399
Customer_zipcode_prefix,-0.030842,-0.004303,0.041878,0.225188,-0.031319,0.056723,0.049997,0.001849,1.000000,0.071668,1.000000,0.115885,-0.312244,0.071668,-0.011225,-0.016091,0.489961,0.010590,0.275780,0.297396,0.060686
Seller_zipcode_prefix,0.030551,-0.018655,0.087132,0.147593,0.005551,0.036800,0.067348,0.011210,0.071668,1.000000,0.071668,-0.036958,-0.035919,1.000000,-0.151135,-0.431021,0.172356,0.002455,0.044854,0.048065,0.123175


In [23]:
df = temp[["Product_id","Review_score","Price","Freight_value","time_diff_purchase_customer_second","time_diff_customer_delivery_second","Distance","Payment_value"]]

In [24]:
product_id_counts = df['Product_id'].value_counts()

In [25]:
product_id_up_2 = product_id_counts[product_id_counts >= 2].index

In [26]:
choose_product_id = df[df['Product_id'].isin(product_id_up_2)]

In [27]:
choose_product_id[choose_product_id.Product_id=="PRODUCT_01923"]

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
100532,PRODUCT_01923,3.0,18.99,18.13,1735166.0,-253285.0,4.075219,82.38
100533,PRODUCT_01923,3.0,18.99,18.13,1735166.0,-253285.0,4.075219,28.89
100534,PRODUCT_01923,3.0,18.99,18.13,1735166.0,-253285.0,4.075219,8.98


In [28]:
product_id_no_dup = choose_product_id.drop_duplicates()

In [29]:
# unique_rows.Product_id.value_counts()

In [30]:
product_id_counts1 = product_id_no_dup['Product_id'].value_counts()

In [31]:
product_id_up_2_1 = product_id_counts1[product_id_counts1 >= 2].index

In [32]:
choose_product_id_1 = product_id_no_dup[product_id_no_dup['Product_id'].isin(product_id_up_2_1)]

In [33]:
dacon_last_df = choose_product_id_1.sort_values(by='Product_id')

In [34]:
# temp.columns

In [35]:
dacon_last_df.columns


Index(['Product_id', 'Review_score', 'Price', 'Freight_value',
       'time_diff_purchase_customer_second',
       'time_diff_customer_delivery_second', 'Distance', 'Payment_value'],
      dtype='object')

In [36]:
variable_importance = {
    'Payment_value': 1,
    'Review_score': 0.8,
    'Price': 0.6,
    'time_diff_customer_delivery_second': 0.5,
    'Distance': 0.4,
    'Freight_value': 0.3,
    'time_diff_purchase_customer_second': 0.2,
}

In [37]:
total_importance = sum(variable_importance.values())

In [38]:
normalized_importance = {key: value / total_importance for key, value in variable_importance.items()}

In [39]:
normalized_importance

{'Payment_value': 0.2631578947368421,
 'Review_score': 0.2105263157894737,
 'Price': 0.15789473684210525,
 'time_diff_customer_delivery_second': 0.13157894736842105,
 'Distance': 0.10526315789473685,
 'Freight_value': 0.07894736842105263,
 'time_diff_purchase_customer_second': 0.052631578947368425}

In [40]:
dacon_last_df.Product_id.unique()

array(['PRODUCT_00005', 'PRODUCT_00006', 'PRODUCT_00009', ...,
       'PRODUCT_29466', 'PRODUCT_29468', 'PRODUCT_29469'], dtype=object)

In [56]:
test_df = dacon_last_df[dacon_last_df.Product_id=="PRODUCT_29468"]

In [57]:
test_df

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
47605,PRODUCT_29468,5.0,127.5,17.14,786643.0,1144294.0,3.012221,144.64
597,PRODUCT_29468,5.0,127.5,11.91,142457.0,1086465.0,0.649095,139.41
52932,PRODUCT_29468,4.0,127.5,11.91,434148.0,1070669.0,0.204524,139.41
44554,PRODUCT_29468,5.0,127.0,12.39,1129132.0,608472.0,6.256644,139.39
39302,PRODUCT_29468,4.0,127.0,34.69,2753075.0,-76352.0,16.906066,161.69


In [ ]:
# 0    0.339522
# 1    0.038950
# 2   -0.386102
# 3   -0.262439
# 4    0.270068

In [59]:
test_df = test_df.drop(["Product_id"],axis=1)

In [61]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standardized = scaler.fit_transform(test_df)
X_standardized_df = pd.DataFrame(X_standardized, columns=test_df.columns)
print(X_standardized_df.Price)

0    0.816497
1    0.816497
2    0.816497
3   -1.224745
4   -1.224745
Name: Price, dtype: float64


In [62]:

for column, weight in normalized_importance.items():
    X_standardized_df[column] *= weight

In [63]:
# print(X_standardized_df)
weighted_sum = X_standardized_df.sum(axis=1)

In [64]:
weighted_sum

0    0.339522
1    0.038950
2   -0.386102
3   -0.262439
4    0.270068
dtype: float64

In [47]:
weighted_sum.max()

0.33952241169598857

In [48]:
# Payment_value' 1등 
# 'Review_score', 2등  
# 'Price', ' 3등  
# 'time_diff_customer_delivery_second',  4등  
# Distance', ' 5등 	 
# Freight_value', 6등 
# time_diff_purchase_customer_second', 7등

In [49]:
#전체다시 합할 df
total_df=pd.DataFrame()
# 가중치
# relative_importance
# 비중 normalized_importance
# test_df = sorted_data[sorted_data.Product_id=="PRODUCT_29466"]
# test_df = test_df.drop(["Product_id"],axis=1)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standardized = scaler.fit_transform(test_df)
X_standardized_df = pd.DataFrame(X_standardized, columns=test_df.columns)
# print(X_standardized_df.Price)
# X_standardized_df
for column, weight in normalized_importance.items():
    X_standardized_df[column] *= weight
# print(X_standardized_df)
weighted_sum = X_standardized_df.sum(axis=1)
# weighted_sum

In [50]:
dacon_last_id = dacon_last_df['Product_id'].unique()

In [51]:
len(dacon_last_id) #12517개

12517

In [52]:
dacon_last_df.head(2)

,Product_id,Review_score,Price,Freight_value,time_diff_purchase_customer_second,time_diff_customer_delivery_second,Distance,Payment_value
67595,PRODUCT_00005,2.0,958.0,27.76,468466.0,1560059.0,4.628797,985.76
64159,PRODUCT_00005,5.0,968.0,44.30,1151056.0,1314046.0,7.932873,1012.30


In [53]:
# dacon_last_df 원본파일
dacon_remake_df = dacon_last_df.copy()

In [54]:
dacon_last_df.Payment_value.sum() 
# 11288914.88

11288914.88

In [55]:
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
i =0
for product_id in tqdm(dacon_last_id):
    test_df = dacon_last_df[dacon_last_df['Product_id'] == product_id]
    test_df = test_df.drop(["Product_id"],axis=1)
    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(test_df)
    X_standardized_df = pd.DataFrame(X_standardized, columns=test_df.columns)
    for column, weight in normalized_importance.items():
        X_standardized_df[column] *= weight
    weighted_sum = X_standardized_df.sum(axis=1)
    weighted_sum_max=weighted_sum.max()
    max_index = weighted_sum.idxmax()
    max_payment_value=test_df.iloc[max_index]["Payment_value"]
    dacon_remake_df.loc[dacon_remake_df["Product_id"]==product_id , "Payment_value"] = max_payment_value

  1%|▋                                                                              | 109/12517 [00:01<03:15, 63.33it/s]


KeyboardInterrupt: 

In [ ]:
dacon_last_df[dacon_last_df.Product_id=="PRODUCT_00032"]
#화요일날이 왜 수요가많나요

In [ ]:
dacon_remake_df[dacon_remake_df.Product_id=="PRODUCT_00032"]

In [ ]:
dacon_remake_df.Product_id.unique().tolist() 
# PRODUCT_00032 PRODUCT_00034 PRODUCT_00035

In [ ]:
print(i)

In [ ]:
dacon_remake_df.Payment_value.sum() 
# 19173054.28

In [ ]:
dacon_remake_df.Payment_value.sum()  - dacon_last_df.Payment_value.sum()

In [ ]:
dacon_last_df.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
dacon_remake_df.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)